In [1]:
import spacy
import pickle
import random

In [2]:
!git clone https://github.com/laxmimerit/CV-Parsing-using-Spacy-3.git

Cloning into 'CV-Parsing-using-Spacy-3'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 82 (delta 14), reused 74 (delta 14), pack-reused 6
Receiving objects: 100% (82/82), 5.62 MiB | 6.53 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [3]:
train_data = pickle.load(open('/content/CV-Parsing-using-Spacy-3/data/training/train_data.pkl', 'rb'))

In [5]:
train_data[190]

('Pawan Nag Microsoft Certified System Engineer  Delhi, Delhi - Email me on Indeed: indeed.com/r/Pawan-Nag/e14493f28cb72022  WORK EXPERIENCE  Microsoft Certified System Engineer  MCSA -  2000 to 2000  Corporate Trainings in India  EDUCATION  AMIE sec A  Training Program  2018  LINKS  https://mcp.microsoft.com/Anonymous//Transcript/Validate  ADDITIONAL INFORMATION  Skill Summary: Operating Systems: • Windows Server 2000, 2003, 2008, 2012, 2012R2. • Exchange 2003, 2007, 2010, 2013,  Date […] (Name) Pawan Nag  https://www.indeed.com/r/Pawan-Nag/e14493f28cb72022?isid=rex-download&ikw=download-top&co=IN https://mcp.microsoft.com/Anonymous//Transcript/Validate',
 {'entities': [(523, 562, 'Email Address'),
   (500, 509, 'Name'),
   (380, 398, 'Skills'),
   (240, 250, 'Degree'),
   (150, 175, 'Designation'),
   (140, 149, 'Designation'),
   (82, 121, 'Email Address'),
   (54, 59, 'Location'),
   (47, 52, 'Location'),
   (20, 45, 'Designation'),
   (10, 19, 'Designation'),
   (0, 9, 'Name')]})

In [8]:
nlp = spacy.blank('en')

def train_model(train_data):
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe('ner', last = True)

    for _, annotation in train_data:
        for ent in annotation['entities']:
            ner.add_label(ent[2])


    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(10):
            print("Statring iteration " + str(itn))
            random.shuffle(train_data)
            losses = {}
            index = 0
            for text, annotations in train_data:
                try:
                    nlp.update(
                        [text],  # batch of texts
                        [annotations],  # batch of annotations
                        drop=0.2,  # dropout - make it harder to memorise data
                        sgd=optimizer,  # callable to update weights
                        losses=losses)
                except Exception as e:
                    pass

            print(losses)


In [9]:
train_model(train_data)

Statring iteration 0
{}
Statring iteration 1
{}
Statring iteration 2
{}
Statring iteration 3
{}
Statring iteration 4
{}
Statring iteration 5
{}
Statring iteration 6
{}
Statring iteration 7
{}
Statring iteration 8
{}
Statring iteration 9
{}


In [10]:
nlp.to_disk('nlp_model')

In [11]:
nlp_model = spacy.load('nlp_model')


In [12]:
train_data[0][0]

'Soumya Balan IT SUPPORT  Sulthan Bathery, Kerala, Kerala - Email me on Indeed: indeed.com/r/Soumya- Balan/97ead9542c575355  ➢ To work in a progressive organization where I can enhance my skills and learning to contribute to the success of the organization.  WORK EXPERIENCE  Technical support engineer  Microsoft  Position: TECHNICAL SUPPORT ENGINEER  Company: Microsoft Corporation - Microsoft India Global Technical Support Center (Microsoft IGTSC), Bangalore  Years of Experience: 2 Years and 4 Months  Responsibilities  ➢ Represent Microsoft and communicate with corporate customers via telephone, written correspondence, or electronic service regarding technically complex escalated problems identified in Microsoft software products, and manage relationships with those customers.  ➢ Manage not only the technically complex problems, but also politically charged situations requiring the highest level of customer skill.  ➢ Receive technically complex, critical or politically hot customer iss

In [14]:
doc = nlp_model(train_data[0][0])
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')